In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
os.system('clear')

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train.head()

In [ ]:

y_train = train['label'].astype('float32')
X_train = train.drop(['label'], axis=1).astype('int32')
X_test = test.astype('float32')
X_train.shape, y_train.shape, X_test.shape

In [ ]:
sns.countplot(x='label', data=train)

In [ ]:
# Data normalization
X_train = X_train/255
X_test = X_test/255
X_train = X_train.values.reshape(-1,28,28,1)
X_test = X_test.values.reshape(-1,28,28,1)
X_train.shape, X_test.shape

In [ ]:
# one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes = 10)
y_train.shape

In [ ]:
print(train['label'].head())
y_train[0:5,:]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X_train, y_train, test_size = 0.1, random_state=42)
plt.imshow(X_train[1][:,:,0])
plt.title(y_train[1].argmax())

In [ ]:
# Building a CNN model
input_shape = (28,28,1)
X_input = tf.keras.layers.Input(input_shape)

# layer 1
x = tf.keras.layers.Conv2D(64,(3,3),strides=(1,1),name='layer_conv1',padding='same')(X_input)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D((2,2),name='maxPool1')(x)
# layer 2
x = tf.keras.layers.Conv2D(32,(3,3),strides=(1,1),name='layer_conv2',padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D((2,2),name='maxPool2')(x)
# layer 3
x = tf.keras.layers.Conv2D(32,(3,3),strides=(1,1),name='conv3',padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling2D((2,2), name='maxPool3')(x)
# fc
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64,activation ='relu',name='fc0')(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(32,activation ='relu',name='fc1')(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(10,activation ='softmax',name='fc2')(x)

conv_model = tf.keras.models.Model(inputs=X_input, outputs=x, name='Predict')
conv_model.summary()

In [ ]:
# Adam optimizer
conv_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.fit(X_train, y_train, epochs=10, batch_size=100, validation_data=(X_cv,y_cv))

In [ ]:
# SGD optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=0.0005, ema_momentum=0.5, weight_decay=0.0, nesterov=False) 
conv_model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.fit(X_train, y_train, epochs=12, validation_data=(X_cv, y_cv))

In [ ]:
y_pred = conv_model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
my_submission = pd.DataFrame({'ImageId': list(range(1, len(y_pred)+1)), 'Label': y_pred})
my_submission.to_csv('submission.csv', index=False)